In [14]:
from cornac.data import Reader
from cornac.eval_methods import RatioSplit
from cornac.models import MF
import pandas as pd
import numpy as np

# Cargar dataset
reader = Reader()
data = reader.read(fpath='datasets/ml-100k/u.data', fmt='UIRT', sep='\t')

# Separación entrenamiento/test
eval_method = RatioSplit(
    data=data,
    test_size=0.2,
    rating_threshold=0.0,
    exclude_unknowns=True,
    verbose=True,
    seed=42
)

# Entrenar modelo
model = MF(k=10, max_iter=50, learning_rate=0.01, verbose=True)
model.fit(eval_method.train_set)

# Historial de ítems vistos por usuario
train_set = eval_method.train_set
user_rated = {}

user_indices, item_indices, _ = train_set.uir_tuple

for uid, iid in zip(user_indices, item_indices):        # user_indices, item_indices son arrays paralelos → zip itera sobre ambos a la vez.
    if uid not in user_rated:
        user_rated[uid] = set()
    user_rated[uid].add(iid)

# Todos los ítems posibles (índices internos)
all_items_internal = set(train_set.iid_map.values())


rating_threshold = 0.0
exclude_unknowns = True
---
Training data:
Number of users = 943
Number of items = 1651
Number of ratings = 80000
Max rating = 5.0
Min rating = 1.0
Global mean = 3.5
---
Test data:
Number of users = 943
Number of items = 1651
Number of ratings = 19964
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 943
Total items = 1651


  0%|          | 0/50 [00:00<?, ?it/s]

Optimization finished!


In [7]:
TOP_N = 5

def get_top_n(model, train_set, user_rated, all_items_internal, name="MF"):
    top_n = {}
    for uid in train_set.uid_map.values():
        seen = user_rated.get(uid, set())
        candidates = all_items_internal - seen

        scores = [(iid, model.score(uid, iid)) for iid in candidates]
        scores.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = [iid for iid, _ in scores[:TOP_N]]
        
    print(f"Top-{TOP_N} generado para modelo {name}")
    return top_n

top_n = get_top_n(model, train_set, user_rated, all_items_internal)

Top-5 generado para modelo MF


In [8]:
# La popularidad de un ítem se puede medir como la cantidad de veces que aparece en el set de entrenamiento. Es decir, cuántos usuarios lo han calificado.

from collections import Counter

# Contar cuántas veces aparece cada ítem en el entrenamiento
item_counts = Counter(item_indices)

# Normalizar popularidad a valores entre 0 y 1
max_count = max(item_counts.values())
item_popularity = {iid: count / max_count for iid, count in item_counts.items()}

# Mostrar ejemplo
sorted_pop = sorted(item_popularity.items(), key=lambda x: x[1], reverse=True)
print("Ítems más populares (top 5):")
for iid, pop in sorted_pop[:5]:
    print(f"Ítem {iid} → Popularidad: {pop:.4f}")



Ítems más populares (top 5):
Ítem 92 → Popularidad: 1.0000
Ítem 305 → Popularidad: 0.8981
Ítem 212 → Popularidad: 0.8684
Ítem 76 → Popularidad: 0.8599
Ítem 188 → Popularidad: 0.8365


In [ ]:
def promedio_popularidad(top_n, item_popularity):
    total = 0
    count = 0
    for recs in top_n.values():
        for iid in recs:
            total += item_popularity.get(iid, 0)
            count += 1
    return total / count if count > 0 else 0

pop_promedio = promedio_popularidad(top_n, item_popularity)
print(f" Popularidad promedio de las recomendaciones: {pop_promedio:.4f}")


🎯 Popularidad promedio de las recomendaciones: 0.2877


In [11]:
def get_top_n_reranked(model, train_set, user_rated, all_items, item_popularity, top_n=5, lambda_=0.5):
    reranked_top_n = {}

    for uid in train_set.uid_map.values():
        seen = user_rated.get(uid, set())
        candidates = all_items - seen

        penalized_scores = []
        for iid in candidates:
            score = model.score(uid, iid)
            popularity = item_popularity.get(iid, 0)
            penalized_score = score - lambda_ * popularity
            penalized_scores.append((iid, penalized_score))

        penalized_scores.sort(key=lambda x: x[1], reverse=True)
        reranked_top_n[uid] = [iid for iid, _ in penalized_scores[:top_n]]

    print(f"Top-{top_n} con penalización generada (λ={lambda_})")
    return reranked_top_n


lambda_penalizacion = 0.5
top_n_rerank = get_top_n_reranked(
    model, train_set, user_rated, all_items_internal,
    item_popularity, top_n=5, lambda_=lambda_penalizacion
)

pop_rerank = promedio_popularidad(top_n_rerank, item_popularity)
print(f"Popularidad promedio (reranked): {pop_rerank:.4f}")


Top-5 con penalización generada (λ=0.5)
Popularidad promedio (reranked): 0.1683


In [ ]:
# Calcular Precision@5 para ambos conjuntos de recomendaciones
from cornac.metrics import Precision

# Crear mapa item_id -> item_index interno
item_id2index = {v: k for k, v in train_set.iid_map.items()}

# Crear ground truth del test set
test_set = eval_method.test_set
ground_truth = {}

for uid, iid, _ in zip(*test_set.uir_tuple):
    ground_truth.setdefault(uid, set()).add(iid)

# Precision@5
def precision_at_k(top_n, ground_truth, k=5):
    precisiones = []
    for uid, recs in top_n.items():
        gt = ground_truth.get(uid, set())
        if not gt:
            continue
        hits = sum([1 for iid in recs if iid in gt])
        precisiones.append(hits / k)
    return np.mean(precisiones)

precision_original = precision_at_k(top_n, ground_truth)
precision_rerank = precision_at_k(top_n_rerank, ground_truth)

print(f"Popularidad promedio original: {pop_promedio:.4f}")
print(f"Popularidad promedio reranjed: {pop_rerank:.4f}")


print(f"Precision@5 original: {precision_original:.4f}")
print(f"Precision@5 rerankeado: {precision_rerank:.4f}")

 Popularidad promedio original: 0.2877
Popularidad promedio reranjed: 0.1683
Precision@5 original: 0.0836
Precision@5 rerankeado: 0.0472


**Tabla comparativa de resultados**

| Métrica              | Recomendación Original | Recomendación Rerankeada |
| ---------------------| ---------------------- | ------------------------ |
| Precision\@5         | 0.0836                 | 0.0472                   |
| Popularidad Promedio | 0.2877                 | 0.1683                   |

**El modelo original prioriza ítems populares**

* La popularidad promedio de 0.2877 indica que el modelo MF tiende a recomendar los ítems más frecuentemente vistos.
* Este comportamiento es común en modelos entrenados solo con historial de interacción, sin ninguna corrección.

**El re-ranking penaliza la popularidad efectivamente**

* Con `λ = 0.5`, la popularidad promedio bajó a 0.1683.
* Esto significa que el sistema ahora recomienda ítems más variados y menos populares, mejorando la equidad en la exposición.

**La precisión disminuyó**

* La `Precision@5` bajó de 0.0836 a 0.0472.
* Esto refleja un trade-off inevitable: al diversificar, se pierde algo de precisión.